Written by:
- Joshua Wylie

Edited by:
- Witold Nazarewicz

# Introduction

This notebook will focus on the concepts and fundamentals of $\chi^2$ minimization. For an in-depth review see [J Dobaczewski et al 2014 J. Phys. G: Nucl. Part. Phys. 41 074001](https://iopscience.iop.org/article/10.1088/0954-3899/41/7/074001).

$\chi^2$ minimization is a specific case of so-called optimization problems. These problems focus on using some convex function to force a minimum to appear in the function itself, thereby allowing us to identify important features from the parameters of the function. In other words, optimization problems use a special set of functions like $\chi^2$ (shown below) to generate a dimensionless quantity that can be used to identify important aspects between theoretical models and experimental data. By creating a singular quantity which serves as a measure of how good the model matches the data, we can systematically find points where a minimum would correspond to the "best fit" of the model to the data. You have likely encountered similar concepts before under different names like *linear regression*, *ridge regression*, or discussions on *fitting* procedures. This is reflected in the $\chi^2$ function
$$
\chi^2(\vec{p}) = \sum_{i=1}^{N_d} \frac{\left(O_i(\vec{p}) - O_{i}^{\rm exp}\right)^2}{\Delta O_i^2}
$$
where $\vec{p}$ is a vector containing the paremeters, e.g. $\vec{p}=(p_0, p_1,...,p_n)$ for distinct parameters $p_i$, $O_i(\vec{p})$ is the $i$-th model or theoretically calculated value, $O_i^{\rm exp}$ is the $i$-th  experimental data point, and $\Delta O_i$ is the $i$-th adopted error.

Generally these (penalty) functions can be chosen or tailored to your specific problem. In many cases, using $\chi^2$ is a decent starting point. We will elaborate more on the inner workings of the process as we go, but there are a few general ideas to remember as we go through:

1. What type of data do you have? In some cases, it might be better to consider different forms of the data, like taking a log for data which varies by orders of magnitiude.

2. The error term $\Delta O_i$ is not *only* the experimental error, but a sum of $\Delta O_i^2 = (\Delta O_i^{\rm exp})^2 + (\Delta O_i^{\rm num})^2 + (\Delta O_i^{\rm the})^2$ for the experimental, numerical, and theoretical errors respectively.

3. When we refer to "residuals" these are the differences between experimental and calculated theoretical (model) value.

4. By using a dimensionless function like $\chi^2$, we can use a variety of observables for each datapoint $i$ as we will calculate a difference between theory and experimental value $\left(O_i(\vec{p}) - O_{i}^{\rm exp}\right)^2$ divided by the same units from the error $\Delta O_i^2$ resulting in a cancelation of units.

5. The error $\Delta O_i^2$ can be somewhat arbitrary due to the dependence on user input. Regardless, you can consider this as a weight term by $W_i = 1 / \sqrt{\Delta O_i}$.

If you're interested in more statistics information and explanations on why we do things, please take a look at videos on the [ASCSN YouTube page](https://youtube.com/@ascsn-channel?si=m5T-JFMKyQsiSeX5) for content like a guide to [practical Bayesian statistics](https://youtu.be/YdxzBLkFPJA?si=q63BN3uRO_Mb0xMQ).

Before we get started, we'll import some (hopefully) familiar packages that we'll use a lot later. If these are unfamiliar, please see the [Introduction to Python Packages](https://github.com/ascsn/online-guides/blob/main/Coding/Python%20Packages.ipynb) notebook and the previous [assignment on the ASCSN GitHub](https://github.com/ascsn/online-guides/blob/main/Coding/introduction_to_reading_data_and_plotting.ipynb).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Getting the dataset

We assume that you are following along with or participating in the Nuclear Structure Course (PHY981) offered at Michigan State University or a similar course at your current institution. If you are not, a previous assignment titled ["introduction_to_reading_data_and_plotting.ipynb"](https://github.com/ascsn/online-guides/blob/main/Coding/introduction_to_reading_data_and_plotting.ipynb) focused on generating the data set we will be using. We recommend you take a look at that notebook now or at minimum the "Assignment" section.

Now, we can read the data from the file we created in the previous assignment (should be a file named ```assignment_output_for_reading_data_and_plotting.csv```).

In [ ]:
dataset = pd.read_csv('assignment_output_for_reading_data_and_plotting.csv')
display(dataset)

Let's take a look at our data now using Matplotlib.

In [ ]:
# Use the same code as before...
fig, ax = plt.subplots(figsize=[5,5]) # Figure and axis subplot panel respectively

# We will plot data directly onto the axis panel
ax.errorbar(
    dataset['x'], dataset['y'], yerr=2*dataset['error'], # plot x, y, and error data
    fmt='o', capsize=7, # set the format to be circular data points and error bar width size
    label='Data' # Note that we now include a label to go with our legend later
)

# Set the x and y-labels for the plot
ax.set_xlabel('x',fontsize=16)
ax.set_ylabel('y',fontsize=16)

# Set x and y limits
ax.set_xlim(0, 1)
ax.set_ylim(-2,4)

# Set title
ax.set_title('Example data plot', fontsize=16)

# Include a legend on the given axis to the top left quadrant
ax.legend(loc='upper left')

# Setup and Calculate $\chi^2$ minimization

In the following we will create our $\chi^2$ python function and use the package ```scipy``` which has a lot of useful functions like ```optimize.minimize``` to minimize a given function. Before we go any further, we'll make sure this is setup using the cells below.

In [ ]:
!pip3 install scipy

In [5]:
from scipy.optimize import minimize

## Picking the model

In the introduction, we hinted at a key idea which we will expand upon now. All of this minimization discussion is centered around a model we pick. Let's imagine we don't know the form of the random data above and are left to come up with our own model. It seems to have $y$ increase proportionally with $x$ so perhaps a good model for this would be a linear one like
$$
y = mx + b.
$$
Of course, if we knew something about the data, maybe we would decide a linear one is not the best and pick another. For now, we'll assume we have no underlying knowledge about the data form and make our linear function in ```python```.

In [6]:
def linear_model(x, m, b):
    '''
    Function returning the calculated value of a simple line. Given slope m and intercept b, calculates and returns the linear value y.
    '''
    return m * x + b

Now that we have a function, we will give an example of its use in relation to the data. Since we've picked a model based on a presumed form of the data (that is seems linear), we might want to see if this holds up. For illustration sake, we pick a random set of parameters below.

In [ ]:
# Set initial parameters for m and b respectively
initial_parameters = [2, -0.25]

# Calculate the value for our model given the initial parameters
modeled_y = linear_model(dataset['x'], initial_parameters[0], initial_parameters[1])

# Use the same code as in the "introduction_to_reading_data_and_plotting.ipynb" notebook...
fig, ax = plt.subplots(figsize=[5,5]) # Figure and axis subplot panel respectively

# We will plot data directly onto the axis panel
ax.errorbar(
    dataset['x'], dataset['y'], yerr=2*dataset['error'], # plot x, y, and error data
    fmt='o', capsize=7, # set the format to be circular data points and error bar width size
    label='Data' # Note that we now include a label to go with our legend later
)

# Now we can create any arbitrary line and plot it
ax.plot(dataset['x'], modeled_y, label='Initial Guess')

# Set the x and y-labels for the plot
ax.set_xlabel('x',fontsize=16)
ax.set_ylabel('y',fontsize=16)

# Set x and y limits
ax.set_xlim(0, 1)
ax.set_ylim(-2,4)

# Set title
ax.set_title('Example data plot', fontsize=16)

# Include a legend on the given axis to the top left quadrant
ax.legend(loc='upper left')

At a quick glance, this doesn't look too bad but it certianly isn't a systematic or rigerous way to pick the parameters. Let's now do our procedure to get the optimized parameters.

## Define our $\chi^2$ function

We will define our $\chi^2$ python function below and aim to keep it as general as possible. In this linear $\chi^2$ function, we will pass-in a set of parameters which are our vector $\vec{p}=\left[p_0, p_1, ..., p_n\right]$, a dataset, and how we might want to represent the theoretical error. In essence our function will go from the general form
$$
\chi^2(\vec{p}) = \sum_{i=1}^{N_d} \frac{\left(O_i(\vec{p}) - O_{i}^{\rm exp}\right)^2}{\Delta O_i^2}
$$
to a more specific, linear, model where $\vec{p}=\left[m, b\right]$ and
$$
\chi^2(\vec{p}) = \sum_{i=1}^{N_d} \frac{\left(O_i(\vec{p}) - O_{i}^{\rm exp}\right)^2}{\Delta O_i^2} = \sum_{i=1}^{N_d} \frac{\left(m x_i + b - O_{i}^{\rm exp}\right)^2}{\Delta O_i^2}.
$$
Note we substituted our linear model in for $O_i(\vec{p})$. Again, we implement this specific case below as a python function. We can always generalize this more, but for simplicity we will leave this as-is.

In [8]:
# Create function for linear chi^2
def linear_chi_square(parameters, data_frame, theoretical_error_type):
    m, b = parameters
    
    # We will get the column data as numpy arrays to more easily handle the data
    experimental_x = data_frame['x'].to_numpy()
    experimental_y = data_frame['y'].to_numpy()
    experimental_error = data_frame['error'].to_numpy()

    # Calculate the theoretical values
    theory_y = linear_model(experimental_x, m, b)

    # Calculate the residuals
    residuals = experimental_y - theory_y

    # Calculate the total errors i.e. O^{exp} + O^{num} + O^{the}
    # Depending on our preference, we can either calculate using no theoretical error or a simple RMSE
    if theoretical_error_type.lower() == 'none':
        theoretical_error = 0
    elif theoretical_error_type.lower() == 'rmse':
        theoretical_error = np.sqrt(np.mean(residuals**2)) # Use RMSE to estimate theoretical error
    
    # calculate total error
    total_error = experimental_error + theoretical_error

    # Calculate the chi^2
    chi_square = (residuals**2) / (total_error**2)

    # Return sum over all chi^2 values
    return np.sum(chi_square)

# Create a function which will also calculate the normalized linear function chi^2
def normalized_linear_chi_square(parameters, data_frame, theoretical_error_type):
    chi_square = linear_chi_square(parameters, data_frame, theoretical_error_type)
    n_parameters = len(parameters)
    n_data = len(data_frame['y'])
    return chi_square / (n_data - n_parameters)

Now that we have our $\chi^2$ function, we can minimize it given our initial parameter guess. Note in the above function, we assumed the form of the total error
$$
\Delta O_i^2 = (\Delta O_i^{\rm exp})^2 + (\Delta O_i^{\rm num})^2 + (\Delta O_i^{\rm the})^2
$$
with
\begin{align}
\Delta O_i^{\rm exp} &= {\rm experimental\; values} \nonumber \\
\Delta O_i^{\rm num} &= 0 \nonumber \\
\Delta O_i^{\rm the} &= \sqrt{\frac{1}{N}\sum_{i}^{N}(O_i(\vec{p}) - O_{i}^{\rm exp})^2}. \nonumber 
\end{align}
The choice of theoretical uncertainties can be modified if desired.

## Minimize $\chi^2$

Now that we have our $\chi^2$ python function, we can use ```scipy.optimize.minimize``` to minimize it given some initial conditions.

In [ ]:
# Perform minimization
minimized_result = minimize(linear_chi_square, initial_parameters, args=(dataset, 'RMSE'))

# Let's look at the optimized parameters
optimized_parameters = minimized_result.x

# We will print only up to the first 3 decimals
print('Optimized m = {:.3f}'.format(optimized_parameters[0]))
print('Optimized b = {:.3f}'.format(optimized_parameters[1]))

# We can also print the minimized chi^2 value
minimum_chi_square = minimized_result.fun

print("Minimum chi^2 value = {:.3f}".format(minimum_chi_square))

Let's now look at our minimized result! We will also consider regions outside our dataset for extrapolation.

In [ ]:
# Set an analytical x domain beyond our dataset just so we can get a feel for how it may extrapolate
x = np.linspace(0, 1.5, 100)

# Calculate the value for our model given the initial parameters
modeled_y = linear_model(x, optimized_parameters[0], optimized_parameters[1])

# Use the same code as in the "introduction_to_reading_data_and_plotting.ipynb" notebook...
fig, ax = plt.subplots(figsize=[5,5]) # Figure and axis subplot panel respectively

# We will plot data directly onto the axis panel
ax.errorbar(
    dataset['x'], dataset['y'], yerr=2*dataset['error'], # plot x, y, and error data
    fmt='o', capsize=7, # set the format to be circular data points and error bar width size
    label='Data' # Note that we now include a label to go with our legend later
)

# Now we can create any arbitrary line and plot it
ax.plot(x, modeled_y, label='Optimized Parameters')

# Set the x and y-labels for the plot
ax.set_xlabel('x',fontsize=16)
ax.set_ylabel('y',fontsize=16)

# Set x and y limits
# ax.set_xlim(0, 1)
ax.set_ylim(-2,4)

# Set title
ax.set_title('Example data plot', fontsize=16)

# Include a legend on the given axis to the top left quadrant
ax.legend(loc='upper left')

Ultimately, this doesn't look much more impressive compared to our original guess, so why bother with a process like this? Well, the point of minimization isn't only to get a nice fitting line to our data, but to also be able to say something more concrete about why its a good calibration! We can also see how the $\chi^2$ space changes with varying values of $m$ and $b$ in the 3D plot below. This is just to highlight the minimum in the parameter space, and statistical considerations will be elaborated on more below.

In [ ]:
# Define the mesh grid and parameter range for m and b
m_range = np.linspace(-5, 5, 100)  # Range for slope m
b_range = np.linspace(-5, 5, 100)  # Range for intercept b

# Meshgrid will make a 2D array 
M, B = np.meshgrid(m_range, b_range)

# Calculate chi-squared for each combination of m and b
chi_squared_values = np.zeros_like(M)

for i in range(M.shape[0]):
    for j in range(M.shape[1]):
        params = (M[i, j], B[i, j])
        chi_squared_values[i, j] = linear_chi_square(params, dataset, theoretical_error_type='rmse')

# Replotting the contour with the minimum point highlighted
plt.figure(figsize=(10, 8))
contour = plt.contourf(M, B, chi_squared_values, levels=50, cmap='viridis')
plt.colorbar(contour, label=r'$\chi^2$')

# Highlight the minimum point
plt.plot(optimized_parameters[0], optimized_parameters[1], marker='*', color='gold', markersize=15, label='Minimum $\\chi^2$')

# Add labels and title
plt.title(r'$\chi^2$ Contour Plot with Minimum Highlighted', fontsize=16)
plt.xlabel(r'Slope $m$', fontsize=14)
plt.ylabel(r'Intercept $b$', fontsize=14)
plt.legend(fontsize=12)
plt.grid(alpha=0.3)
plt.show()


## Statistical Considerations

As we discussed above, there are additional tools we can use to make more helpful claims about the fit we just made. One of the first things we'll need is the Hessian Matrix which is used to determine if the convex function we optimized has a local minima.

### Calculating the Hessian

The Hessian can be calculated (as shown in [J Dobaczewski et al 2014 J. Phys. G: Nucl. Part. Phys. 41 074001](https://iopscience.iop.org/article/10.1088/0954-3899/41/7/074001)) by
$$
\mathcal{M}_{\alpha \beta} = \frac{1}{2} \partial_\alpha \partial_\beta \chi^2 |_{\vec{p}_0}
$$
where the partial derivatives are evaluated at the optimized parameter set $\vec{p}_0$. In the case of our linear form, our equation is
$$
\chi^2 = \sum_i \frac{\left(mx_i + b - O_i^{\rm exp}\right)^2}{\Delta O_i^2}
$$
for which we can generate the Hessian
$$
\mathcal{M} = \frac{1}{2} \begin{bmatrix}
\frac{\partial^2}{\partial m^2}\chi^2 & \frac{\partial^2}{\partial m \partial b}\chi^2 \\
\frac{\partial^2}{\partial b \partial m}\chi^2 & \frac{\partial^2}{\partial b^2}\chi^2
\end{bmatrix} = \frac{1}{2} \begin{bmatrix}
\sum_i \frac{2x_i^2}{\Delta O_i^2} & \sum_i \frac{2x_i}{\Delta O_i^2} \\
\sum_i \frac{2x_i}{\Delta O_i^2} & \sum_i \frac{2}{\Delta O_i^2}
\end{bmatrix} = \begin{bmatrix}
\sum_i \frac{x_i^2}{\Delta O_i^2} & \sum_i \frac{x_i}{\Delta O_i^2} \\
\sum_i \frac{x_i}{\Delta O_i^2} & \sum_i \frac{1}{\Delta O_i^2}
\end{bmatrix}.
$$
This can be done analytically as shown and there are other packages like [```numdifftools```](https://numdifftools.readthedocs.io/en/master/), [```autograd```](https://github.com/HIPS/autograd/tree/7c22772cb42455c00dac964c5363242e62529ed6), or any other finite difference solver (shown in the Extras section) which can do it for you automatically. We also show how one can do it with ```Sympy``` (a symbolic package) later in the "Extras" section.

The important thing is that we first calculate our Hessian to then get the Covariance Matrix. We use the derived form above for the Hessian and implement the calculations via ```numpy``` array operations.

In [ ]:
# Calculate the RMSE for the theoretical error like we did in linear_chi_squared (or set to zero if you decided not to do this)
rmse = np.sqrt(np.mean(
    (dataset['y'] - linear_model(dataset['x'], optimized_parameters[0], optimized_parameters[1]))**2
)) # Use RMSE to estimate theoretical error

# Alternative comment the above rmse value and uncomment below to set to zero
# rmse = 0

# Create Hessian matrix first as a 2x2 array of zeros
manual_hessian_matrix = np.zeros((2,2), dtype=float)

# Calculate first diagonal term i.e. partial d^2 / dm^2
manual_hessian_matrix[0,0] = np.sum(dataset['x']**2 / (dataset['error'] + rmse)**2)

# Calculate the off-diagonals
manual_hessian_matrix[0,1] = np.sum(dataset['x'] / (dataset['error'] + rmse)**2)
manual_hessian_matrix[1,0] = np.sum(dataset['x'] / (dataset['error'] + rmse)**2)

# Calculate the second diagonal i.e. partial d^2 / db^2
manual_hessian_matrix[1,1] = np.sum(1 / (dataset['error'] + rmse)**2)

# Show our calculated Hessian
print('Analytically calculated Hessian =')
display(manual_hessian_matrix)

### Calculating the Covariance Matrix

Now that we have our Hessian $\mathcal{M}$ we can calculate the covariance matrix $\mathcal{C}$ via
$$
\mathcal{C} = s \mathcal{M}^{-1}
$$
where $s$ is
$$
s = \frac{\chi^2(\vec{p}_0)}{N_d - N_p}.
$$
$N_d$ and $N_p$ are the number of observables and number of parameters respectively. Since our current case has just a $2\times 2$ matrix, we can analytically calculate the inverse
$$
\begin{bmatrix}
a & b \\
c & d
\end{bmatrix}^{-1} = \frac{1}{ad - bc}
\begin{bmatrix}
d & -b \\
-c & a
\end{bmatrix}.
$$
We calculate the covariance matrix by manually inverting the Hessian
$$
\mathcal{M}^{-1} = \frac{1}{\left(\sum_i \frac{x_i^2}{\Delta O_i^2}\right) \left(\sum_i \frac{1}{\Delta O_i^2}\right) - \left(\sum_i \frac{x_i}{\Delta O_i^2}\right) \left(\sum_i \frac{x_i}{\Delta O_i^2}\right)}
\begin{bmatrix}
\sum_i \frac{1}{\Delta O_i^2} & -\sum_i \frac{x_i}{\Delta O_i^2} \\
-\sum_i \frac{x_i}{\Delta O_i^2} & \sum_i \frac{x_i^2}{\Delta O_i^2}
\end{bmatrix}.
$$
This is implemented with ```numpy``` below.

In [ ]:
# Calculate the determinant of the Hessian (serves as our ad - bc value)
manual_hessian_det = np.linalg.det(manual_hessian_matrix)

# Create a zeros array to store the hessian inverse
manual_hessian_inv = np.zeros_like(manual_hessian_matrix)

# Set the values of the inverse according to the hessian locations
manual_hessian_inv[0,0] = manual_hessian_matrix[1,1]
manual_hessian_inv[0,1] = -manual_hessian_matrix[0,1]
manual_hessian_inv[1,0] = -manual_hessian_matrix[1,0]
manual_hessian_inv[1,1] = manual_hessian_matrix[0,0]

# Scale by dividing by determinant
manual_hessian_inv = manual_hessian_inv / manual_hessian_det

print('Analytical inverse of the Hessian =')
display(manual_hessian_inv)

One can also calculate the inverse of a matrix numerically via ```numpy``` using the ```linalg.inv``` function. We note that ```linalg``` contains very helpful linear algebra functions which are fast and fairly user-friendly, so try to use these instead of coding your own!

In [ ]:
print('Numerical inverse of the Hessian =')
display(np.linalg.inv(manual_hessian_matrix))

Now that we have the inverse of our Hessian, we can calculate our covariance matrix below.

In [ ]:
# Calculate s or the normalized chi^2 using the optimal values
manual_covariance_s = normalized_linear_chi_square(optimized_parameters, dataset, 'RMSE')
print('Value of s =', manual_covariance_s)

# Calculate covariance matrix
manual_covariance_matrix = manual_covariance_s * manual_hessian_inv

print('Covariance matrix =')
display(manual_covariance_matrix)

## Correlation Analysis

Now that we have our covariance matrix, we can perform an analysis on the parameters of our model (our straight line). A correlation coefficient can inform us on how a quantity affects a model and highlight any underlying trends in its parameterization. In order to extract these helpful quantities, we must take our covariance matrix and divide by the variable standard deviation (or the diagonal of the covariance matrix). Ultimately we end up with
$$
c_{AB} = \frac{|\overline{\Delta A \Delta B}|}{\sqrt{\overline{\Delta A^2}} \sqrt{\overline{\Delta A^2}}}
$$
or if you prefer to consider it as a matrix of coefficients in our specific (linear case)
$$
c = \begin{bmatrix}
\frac{|\mathcal{C}_{mm}^2|}{\sqrt{\mathcal{C}_{mm}} \sqrt{\mathcal{C}_{mm}}} & \frac{|\mathcal{C}_{mm} \mathcal{C}_{bb}|}{\sqrt{\mathcal{C}_{mm}} \sqrt{\mathcal{C}_{bb}}} \\
\frac{|\mathcal{C}_{bb} \mathcal{C}_{mm}|}{\sqrt{\mathcal{C}_{bb}} \sqrt{\mathcal{C}_{mm}}} & \frac{|\mathcal{C}_{bb}^2|}{\sqrt{\mathcal{C}_{bb}} \sqrt{\mathcal{C}_{bb}}}
\end{bmatrix}
$$

In [16]:
# Get the standard deviations of our covariance matrix parameters along the diagonal
standard_dev = np.sqrt(np.diag(manual_covariance_matrix))

Since we need to divide by the standard deviations element-wise in the array, we can do a fancy trick! ```numpy```'s "outer" function will take two one-dimensional arrays and multiply them as an outer product to make a 2D array for us! For example, we can make the above denominator by doing the outer product
$$
\xi = \begin{bmatrix}
\sqrt{\mathcal{C}_{mm}} \\
\sqrt{\mathcal{C}_{bb}}
\end{bmatrix} \quad {\rm then} \quad
\xi \xi^T = \begin{bmatrix}
\sqrt{\mathcal{C}_{mm}} \sqrt{\mathcal{C}_{mm}} & \sqrt{\mathcal{C}_{mm}} \sqrt{\mathcal{C}_{bb}} \\
\sqrt{\mathcal{C}_{bb}} \sqrt{\mathcal{C}_{mm}} & \sqrt{\mathcal{C}_{bb}} \sqrt{\mathcal{C}_{bb}}
\end{bmatrix}.
$$
This can be implemented in ```numpy``` as shown below.

In [ ]:
# Calculate correlation coefficient matrix using an outer product
manual_correlation_coefficients = np.abs(manual_covariance_matrix) / np.outer(standard_dev, standard_dev)

print('Correlation coefficient matrix =')
display(manual_correlation_coefficients)

The two extremes of correlation coefficients are $c=1$ and $c=0$ where the parameters are completely correlated and fully independent respectively. We see in our case, the parameters on the diagonal are completely correlated. This is a great sanity check as we are considering cases where we are comparing a parameter to itself, so it should be 1! We see in the off-diagonals the correlation is still close to 1 so the $m$ and $b$ term also appear quite correlated.

# Assignment

Now that we've shown how to do $\chi^2$ minimization for a line, we ask that you do the same process for the following function:
$$
y = ax^2 + bx + c
$$
Using the same dataset, please:
1. Write a new python function for the above quadratic equation and a $\chi^2$ function to be optimized (similar to how we did ```linear_model``` and ```linear_chi_square```) and minimize this function to get optimal parameters.

2. Plot the optimized model on the same plot as the dataset. (Optional) Make a contour surface plot to highlight the minimum location.

3. Calculate the Hessian and covariance matrices.

4. Determine the correlation coefficients for this new model.

In [18]:
# Define your own quadratic model

# Define your own quadratic chi^2 model


In [19]:
# Optimize your chi^2 model

In [20]:
# Perform statistical analysis below

## For PHY981 students

If you are doing this notebook in conjunction with PHY981, we would appreciate your input on this activity. If you are willing, please fill out a very short survey ([https://forms.gle/3tmG9iLZ2DZ9sRAdA](https://forms.gle/3tmG9iLZ2DZ9sRAdA)) on this notebook and a pre-survey on the material for the next topic.

# Extras

## Using ```sympy```

We show below how you can use ```sympy``` to perform the analytical calculations similar to how one might use other software like Mathematica. ```sympy``` can be convinient and has many helpful functions if other packages do not have them. That being said, it can be quite slow at times and should be used with caution.

In [ ]:
!pip3 install sympy

Unlike other packages, sympy requires you to import things as needed like Matrices and variable names. There are many alternate ways to do these definitions so we show a very explicit form below. These can always be modified afterwards, but you still need to import them like other functions.

We start with defining a few variables and functions.

In [19]:
from sympy import ordered, Matrix
from sympy import hessian

# We import the symbols we will use in our equation
from sympy.abc import m, b, d, x, y

If we wanted to keep things extra simple, we can just consider a case where we drop the sum in our $\chi^2$ function and just look at one term
$$
\frac{(mx_i + b - y_i)}{\Delta O_i^2} \rightarrow \frac{(mx + b - y)}{d^2}.
$$
We use this in the following ```sympy``` operations where we substituted $\Delta O_i^2$ for $d_i$ just to keep the ```sympy``` notation simpler.

In [ ]:
# Define our chi^2 equation where y is the experimental data and p is a placeholder for our error
eq = (m*x + b - y)**2 / d**2

# Calling the equation name in Jupyter will print it in a nice LaTeX format
eq

In [21]:
# Set the two variables we wish to differentiate
v = [m, b]

In [22]:
# Set a function to calculate the Jacobian which is used to calculate the Hessian
gradient = lambda f, v: Matrix([f]).jacobian(v)

In [ ]:
# print our Jacobian
gradient(eq, v)

In [ ]:
# We can then print the corresponding Sympy-calculated Hessian
hessian(eq, v)

In [ ]:
# We could also try to calculate the inverse of the above Hessian
hessian(eq, v).inv()

Well, that error doesn't appear good, so how do we reconcile the discrepancy between our analytical form and the ```sympy``` version? Thankfully, the answer is simple in that by only considering one term, we are actually not considering the correct matrix to invert! The presence of the zero-valued determinant error is a result of our matrix inversion equation from above as
$$
\begin{bmatrix}
\frac{2x^2}{p^2} & \frac{2x}{p^2} \\
\frac{2x}{p^2} & \frac{2}{p^2}
\end{bmatrix}^{-1} = \frac{1}{\frac{2x^2}{p^2}\frac{2}{p^2} - \frac{2x}{p^2}\frac{2x}{p^2}}
\begin{bmatrix}
\frac{2}{p^2} & -\frac{2x}{p^2} \\
-\frac{2x}{p^2} & \frac{2x^2}{p^2}
\end{bmatrix} = \frac{1}{0}
\begin{bmatrix}
\frac{2}{p^2} & -\frac{2x}{p^2} \\
-\frac{2x}{p^2} & \frac{2x^2}{p^2}
\end{bmatrix}.
$$
If we recall that our actual analytical case is the multiplication of sums, and consider order of operations, we see that
$$
\left(\sum_i \frac{x_i^2}{\Delta O_i^2}\right) \left(\sum_i \frac{1}{\Delta O_i^2}\right) \neq \left(\sum_i \frac{x_i}{\Delta O_i^2}\right) \left(\sum_i \frac{x_i}{\Delta O_i^2}\right).
$$
This means that when we do ```sympy``` calculations, you should always remember that any tricks you do must be properly considered. We highlight this below by looping and printing over each matrix we would consider to make the Hessian properly.

In [ ]:
# We will loop over each row of the dataset and call that set of values ('y', 'x', and 'error')
for i in range(len(dataset)):
    print(hessian(eq, v).subs(# For each call of the hessian function, we will then substitute in the proper values for the 'x' and 'p' variable
        {x:dataset['x'].iloc[i], d:dataset['error'].iloc[i]+rmse}
    ))

In [ ]:
temp = Matrix([[0,0],[0,0]])
for i in range(len(dataset)):
    temp += hessian(eq, v).subs(# For each call of the hessian function, we will then substitute in the proper values for the 'x' and 'p' variable
            {x:dataset['x'].iloc[i], d:dataset['error'].iloc[i]+rmse}
        )

sympy_hessian_matrix = 0.5 * temp
sympy_hessian_matrix

Now that we have properly summed over each term to construct the true Hessian, we can try to invert this matrix.

In [ ]:
sympy_hessian_matrix.inv()

In [ ]:
# Print our manual Hessian inverse below
print(np.linalg.inv(manual_hessian_matrix))

We see here that this total Hessian can now be inverted properly and matches with our analytical case!

If we want to do everything more correctly with ```sympy```, we show the following case where we directly include the sum. Note that using the sum requires the terms ```x[i]``` which indicates a summation subscript for ```sympy``` to consider.

In [31]:
from sympy import Symbol, IndexedBase, symbols, Sum

First, we can show the general form for our solution below.

In [ ]:
# import sympy as sp

# Define any new symbols
x, y, d = symbols('x y d', cls=IndexedBase)
i = Symbol('i', integer=True)

# Define the summation expression
n = Symbol('n', integer=True)  # Number of terms in the summation
expr = Sum((m * x[i] + b - y[i])**2 / d[i]**2, (i, 0, n - 1))

# Expand the summation into a simplified form
expanded_expr = expr.doit()

# Define the number of terms
n_value = len(dataset)

# Create the substitution dictionary
substitutions = {}
for idx in range(n_value):
    substitutions[x[idx]] = dataset['x'].iloc[idx]
    substitutions[y[idx]] = dataset['y'].iloc[idx]
    substitutions[d[idx]] = dataset['error'].iloc[idx] + rmse

# Substitute the values into the expanded expression
evaluated_expr = expanded_expr.subs(substitutions)

# Compute the Hessian matrix with respect to m and b
sympy_hessian_matrix = 0.5 * hessian(evaluated_expr, (m, b))

# Invert the Hessian matrix
sympy_hessian_matrix_inv = sympy_hessian_matrix.inv()

# Display the results
sympy_hessian_matrix_inv.doit()


Now we can include the exact values and calculate the correct values of the Hessian.

In [ ]:
# Define the summation expression
summand = (m * x[i] + b - y[i])**2 / d[i]**2

# Initialize the evaluated summation result
evaluated_sum = 0

# Iterate over the dataset and compute the summation
for idx in range(len(dataset)):
    # Substitute individual data points into the summand
    term = summand.subs({
        x[i]: dataset['x'].iloc[idx],
        y[i]: dataset['y'].iloc[idx],
        d[i]: dataset['error'].iloc[idx] + rmse
    })
    evaluated_sum += term

# Compute the Hessian matrix with respect to m and b
sympy_hessian_matrix = 0.5 * hessian(evaluated_sum, (m, b))

# Invert the Hessian matrix
sympy_hessian_matrix_inv = sympy_hessian_matrix.inv()

# Display the results
print('Hessian Matrix:')
display(sympy_hessian_matrix)
print()

print('Hessian Inverse:')
display(sympy_hessian_matrix_inv)


Here we see that the results are the same as before!

## Using numerical finite difference

One way to take derivatives is through numerical finite difference methods. There are forward, backward, and central methods which are found in many textbooks and [Wikipedia](https://en.wikipedia.org/wiki/Finite_difference#Multivariate_finite_differences). They can also be used in any number of sequential partial derivatives by starting with the first partial derivative approximation for one of our parameters $p_i$
$$
\frac{\partial f}{\partial p_i} \approx \frac{f(p_i + h_i, p_j) - f(p_i - h_i, p_j)}{2h_i}.
$$
From this step, we know that any additional operations, say a second derivative, would be
$$
\frac{\partial}{\partial p_j} \left(\frac{\partial f}{\partial p_i}\right) \approx \frac{\partial}{\partial p_j} \left(\frac{f(p_i + h_i, p_j) - f(p_i - h_i, p_j)}{2h_i}\right) =
$$
$$
\frac{f(p_i + h_i, p_j + h_j) - f(p_i + h_i, p_j - h_j) - f(p_i - h_i, p_j + h_j) + f(p_i - h_i, p_j - h_j)}{4h_i h_j}.
$$
For the case where we have the second-order partial derivative with respect to the same variable
$$
\frac{\partial}{\partial p_i} \left(\frac{\partial f}{\partial p_i}\right) \approx \frac{\partial}{\partial p_i} \left(\frac{f(p_i + h_i, p_j) - f(p_i - h_i, p_j)}{2h_i}\right) =
$$
$$
\frac{f(p_i + h_i, p_j) - 2f(p_i, p_j) + f(p_i - h_i, p_j)}{h_i^2}.
$$
Here every index $i$ and $j$ corresponds to a spacing for the considered parameter; For example, we can set in our problem $p_i = m$ and $p_j=b$.

The step size can be a bit more confusing in our case. In many finite difference discussions, the spacing $h$ is set by the spacing of the points we are taking a derivative with respect to. In our case, we are taking partial derivatives with respect to parameters which are being optimized, so what we can do is instead perturb the parameters about some point (our optimized point). By perturbing around the point, we can see how our function changes giving us a derivative or a response according to the change. We implement this in the following where we use the same $h=h_i=h_j$.

In [ ]:
# Define the cost function or our chi^2 function
def cost_function(params):
    # Set the parameters from a list of params
    m, b = params

    # Calculate the predicted (model) values
    predicted = linear_model(dataset['x'].to_numpy(), m, b)

    # Calculate the residuals
    residuals = predicted - dataset['y'].to_numpy()

    # Calculate the error terms
    error_term = dataset['error'].to_numpy()
    rms = np.sqrt(np.mean(residuals ** 2))
    
    return np.sum((residuals ** 2) / (rms + error_term))

# Finite differences to compute the Hessian
def compute_finite_diff_hessian(f, params, h=[1e-5, 1e-5]):
    n = len(params)  # Number of parameters
    hessian_ = np.zeros((n, n))  # Initialize the Hessian matrix
    
    # Diagonal terms of double partial derivatives
    hessian_[0,0] = (
        f([params[0] + h[0], params[1]]) - 2*f(params) + f([params[0] - h[0], params[1]])
    ) / (h[0]**2) # For twice partial derivative with respect to m
    hessian_[1,1] = (
        f([params[0], params[1] + h[1]]) - 2*f(params) + f([params[0], params[1] - h[1]])
    ) / (h[1]**2) # For twice partial derivative with respect to b

    # Off-diagonal terms of double partial derivatives
    off_diag = (
        f([params[0] + h[0], params[1] + h[1]]) - f([params[0] + h[0], params[1] - h[1]]) -
        f([params[0] - h[0], params[1] + h[1]]) + f([params[0] - h[0], params[1] - h[1]])
    ) / (4 * h[0] * h[1])
    hessian_[0,1], hessian_[1,0] = off_diag, off_diag
    
    return hessian_


# Test the Hessian computation
finite_diff_hessian = 0.5 * compute_finite_diff_hessian(cost_function, optimized_parameters)

print('Our calculated finite difference Hessian:')
print(finite_diff_hessian)
print()

print('Our calculated analytical Hessian:')
print(manual_hessian_matrix)


We can easily notice a few problems with the numerical solution, namely the values between the numerical and analytic Hessian differ. Ultimately, this won't change our answer too much as we see in the calculation of the correlation coefficients below for the Finite Difference method.

In [ ]:
# Calculate covariance matrix
finite_diff_covariance_matrix = manual_covariance_s * np.linalg.inv(finite_diff_hessian)

# Get the standard deviations of our covariance matrix parameters along the diagonal
finite_diff_standard_dev = np.sqrt(np.diag(finite_diff_covariance_matrix))

# Calculate correlation coefficient matrix using an outer product
finite_diff_correlation_coefficients = np.abs(finite_diff_covariance_matrix) / np.outer(finite_diff_standard_dev, finite_diff_standard_dev)

print('Finite Difference correlation coefficient matrix =')
print(finite_diff_correlation_coefficients)
print()

print('Analytic correlation coefficient matrix =')
print(manual_correlation_coefficients)

One key point we can illustrate, and is shared by solutions with finite difference, is that the values one obtains can be dependent on our spacing parameter, in this case $h$. Usually, to get a nicely converged solution, you must pick a spacing that is small enough to represent the infinitesmal nature of a derivative while not being too small to have conflicts with so-called machine precision. Machine precision is the numerical limit a computer can reliably calculate. We show an example of having too large of a step $h$ to a more reasonable value which produces convergence below.

In [ ]:
exponent_array = np.linspace(0, 6, 24, dtype=float)
h_array = 10**(-exponent_array)

iterative_hessian = np.zeros((2, 2, len(h_array)))

for i in range(len(h_array)):
    iterative_hessian[:,:,i] = 0.5 * compute_finite_diff_hessian(cost_function, optimized_parameters, h=[h_array[i], h_array[i]])

plt.plot(np.log10(h_array), iterative_hessian[0,0,:], label=r'$\frac{\partial^2 f}{\partial m^2}$')
plt.plot(np.log10(h_array), iterative_hessian[1,0,:], label=r'$\frac{\partial^2 f}{\partial m \partial b}$')
plt.plot(np.log10(h_array), iterative_hessian[0,1,:], label=r'$\frac{\partial^2 f}{\partial b\partial m}$')
plt.plot(np.log10(h_array), iterative_hessian[1,1,:], label=r'$\frac{\partial^2 f}{\partial b^2}$')
plt.legend()

plt.title('Convergence of Finite Difference Hessian values')
plt.xlabel(r'$\log_{10}(h)$')
plt.ylabel(r'Hessian value')

## Using ```numdifftools```

Now, all of the code we have done above is possible due to the analytic nature or simpler form of the finite difference, so what happens if we have a more complicated problem? Well, thankfully, many people before us have likely struggled with calculating derivatives, Hessians, etc. so there are likely ```python``` packages which will address our problem. One such package which will calculate a Hessian for you is ```numdifftools```. We show how we can calculate the $\chi^2$ Hessian using this package below.

First, make sure we have ```numdifftools``` installed.

In [ ]:
!pip3 install numdifftools

Now we can go ahead and use ```numdifftools``` by importing it.

In [40]:
import numdifftools as nd

```numdifftools``` requires a so-called *lambda* function from ```python```. The purpose of this lambda function is similar to a regular python function. What we will do below is create our $\chi^2$ function including the summation term, the error terms, and indicating which list of variables ```numdifftools``` will need to derivate (which we've called ```mb```). You'll notice this is a list of parameters which will be handled internally by ```numdifftools```.

In [ ]:
# Define the lambda function for the cost function
f = lambda mb: np.sum(
    ((linear_model(dataset['x'].to_numpy(), mb[0], mb[1]) - dataset['y'].to_numpy()) ** 2)
    / (np.sqrt(
        np.mean(
            (linear_model(dataset['x'].to_numpy(), mb[0], mb[1]) - dataset['y'].to_numpy()) ** 2
        )
    ) + dataset['error'].to_numpy())
)

# Compute the Hessian
nd_hessian_matrix = nd.Hessian(f)
nd_hessian_matrix = 0.5 * nd_hessian_matrix(optimized_parameters)

print('Our calculated numdifftools Hessian')
print(nd_hessian_matrix)
print()

print('Our calculated finite difference Hessian:')
print(finite_diff_hessian)

We see that our finite difference code matches with the automatically made ```numdifftools```. This should hopefully be helpful in cases where your Hessian or $\chi^2$ function does not have a nice analytical form.